In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

In [25]:
# Image dimensions
IMG_SIZE = (48, 48)
BATCH_SIZE = 32

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=20, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load train and test datasets
train_generator = train_datagen.flow_from_directory(
    "dataset/fer/train", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_directory(
    "dataset/fer/test", 
    target_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    color_mode="rgb", 
    class_mode="categorical"
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Number of classes: 7


In [26]:
# Load EfficientNetV2-B0 (pretrained on ImageNet)
base_model = EfficientNetV2B0(weights="imagenet", include_top=False, input_shape=(48, 48, 3))

# Freeze base model layers
base_model.trainable = False

# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")  # Output layer
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

# Model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)  │ (None, 2, 2, 1280)     │     5,919,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,084,183 (23.21 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [27]:
EPOCHS = 15

history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=EPOCHS
)

Epoch 1/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 33s 33ms/step - accuracy: 0.2248 - loss: 1.8646 - val_accuracy: 0.2471 - val_loss: 1.8137
Epoch 2/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.2526 - loss: 1.8222 - val_accuracy: 0.2471 - val_loss: 1.8134
Epoch 3/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.2528 - loss: 1.8133 - val_accuracy: 0.2471 - val_loss: 1.8132
Epoch 4/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.2479 - loss: 1.8164 - val_accuracy: 0.2471 - val_loss: 1.8181
Epoch 5/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.2506 - loss: 1.8114 - val_accuracy: 0.2471 - val_loss: 1.8133
Epoch 6/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 31s 34ms/step - accuracy: 0.2530 - loss: 1.8135 - val_accuracy: 0.2471 - val_loss: 1.8146
Epoch 7/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.2474 - loss: 1.8133 - val_accuracy: 0.2471 - val_loss: 1.8137
Epoch 8/15
898/898 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.2508 - loss: 1.8103 - 

In [28]:
# Get train and test accuracies
train_loss, train_acc = model.evaluate(train_generator)
test_loss, test_acc = model.evaluate(test_generator)

print(f"Train Accuracy: {train_acc * 100:.2f}%")
print(f"Test Accuracy: {test_acc * 100:.2f}%")

898/898 ━━━━━━━━━━━━━━━━━━━━ 24s 27ms/step - accuracy: 0.2514 - loss: 1.8089
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.2456 - loss: 1.8198
Train Accuracy: 25.13%
Test Accuracy: 24.71%


In [29]:
model.save("trained_models/efficientnetv2_fer_15_epochs.keras")

In [32]:
# Assuming you have the true labels and predicted labels for both train and test datasets
train_predictions = model.predict(train_generator)
test_predictions = model.predict(test_generator)

# Convert predictions to class labels (if necessary, depends on your model output)
train_predictions_labels = np.argmax(train_predictions, axis=1)
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Assuming your labels are categorical and represented as integers
print("Train Classification Report:")
print(classification_report(train_generator.classes, train_predictions_labels))

print("Test Classification Report:")
print(classification_report(test_generator.classes, test_predictions_labels))

# You can also extract individual metrics:
# For train
train_report = classification_report(train_generator.classes, train_predictions_labels, output_dict=True)
print(f"Train Precision: {train_report['weighted avg']['precision']:.2f}")
print(f"Train Recall: {train_report['weighted avg']['recall']:.2f}")
print(f"Train F1-Score: {train_report['weighted avg']['f1-score']:.2f}")

# For test
test_report = classification_report(test_generator.classes, test_predictions_labels, output_dict=True)
print(f"Test Precision: {test_report['weighted avg']['precision']:.2f}")
print(f"Test Recall: {test_report['weighted avg']['recall']:.2f}")
print(f"Test F1-Score: {test_report['weighted avg']['f1-score']:.2f}")


898/898 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step
Train Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3995
           1       0.00      0.00      0.00       436
           2       0.00      0.00      0.00      4097
           3       0.25      1.00      0.40      7215
           4       0.00      0.00      0.00      4965
           5       0.00      0.00      0.00      4830
           6       0.00      0.00      0.00      3171

    accuracy                           0.25     28709
   macro avg       0.04      0.14      0.06     28709
weighted avg       0.06      0.25      0.10     28709

Test Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       958
           1       0.00      0.00      0.00       111
           2       0.00      0.00      0.00      1024
           3       0.25      1.00      0.40 

/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/ayushidubey/Desktop/College/Semester-6/NNDL/labs/project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use 